In [1]:
%matplotlib inline


In [2]:
import gym
import numpy as np

In [5]:
env_name='FrozenLake8x8-v0'
env = gym.make(env_name)
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [53]:
env.env.P[0]

{0: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 8, 0.0, False)],
 1: [(0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 1, 0.0, False)],
 2: [(0.3333333333333333, 8, 0.0, False),
  (0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)],
 3: [(0.3333333333333333, 1, 0.0, False),
  (0.3333333333333333, 0, 0.0, False),
  (0.3333333333333333, 0, 0.0, False)]}

In [96]:
P = env.env.P
a = P[0]
for s in P:
    for a in P[s]:
        for p,s,r,_ in P[s][a]:
            if r!=0:
                print(p,s,r)
# [P[0][x] for x in P[0]]

0.3333333333333333 63 1.0
0.3333333333333333 63 1.0
0.3333333333333333 63 1.0
0.3333333333333333 63 1.0
0.3333333333333333 63 1.0
0.3333333333333333 63 1.0


In [163]:
def value_iteration(env, gamma=1.0):
    eps = 1e-20
    v = np.zeros(env.nS)
    iters = 10000
    for i in range(iters):
        prev_v = np.copy(v)
        for s in range(env.nS):
            qa = np.zeros(env.nA)
            for a in range(env.nA):
                qa[a] = sum([p*(r+gamma*prev_v[s_]) for p,s_,r,_ in env.P[s][a]])
            v[s] = max(qa)
        if np.sum(np.abs(prev_v-v)) <=eps:
            print('Converged after ', i+1)
            break
    return v

optimal_v = value_iteration(env.env, gamma=1.0)
print(optimal_v)

Converged after  2357
[1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         0.97820163
 0.92643052 0.         0.85661768 0.94623163 0.98207721 1.
 1.         0.9346049  0.80108992 0.47490377 0.6236214  0.
 0.94467761 1.         1.         0.82561308 0.54223433 0.
 0.53934275 0.61118923 0.85195561 1.         1.         0.
 0.         0.16804079 0.38321763 0.44226934 0.         1.
 1.         0.         0.19467347 0.12090475 0.         0.33240114
 0.         1.         1.         0.73155782 0.46311564 0.
 0.27746705 0.5549341  0.77746705 0.        ]


In [164]:

def compute_policy(env,v, gamma = 1.0):
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            for p,s_,r,_ in P[s][a]:
                q_sa[a]+=p*(r+gamma*v[s_])
        policy[s] = np.argmax(q_sa)
    return policy

policy = compute_policy(env.env, optimal_v, gamma =1.0)

print(policy)


[1. 2. 2. 1. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 2. 0. 0. 0. 0. 2. 3. 3. 2.
 0. 0. 0. 1. 0. 0. 2. 2. 0. 3. 0. 0. 2. 1. 3. 2. 0. 0. 0. 1. 3. 0. 0. 2.
 0. 0. 1. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 2. 1. 0.]


In [165]:

def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs,reward, done, _ = env.step(int(policy[obs]))
        total_reward += gamma ** step_idx * reward
        step_idx +=1
        if done:
            break
    return total_reward

run_episode(env, policy, gamma =0.9)


2.318269218083098e-07

In [166]:


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = np.array([run_episode(env, policy, gamma) for _ in range(n)])
    finished_games = scores[scores != 0.0]
    return np.mean(scores), len(finished_games)/len(scores)

evaluate_policy(env,policy,n=1000, gamma = 1.0)
    

(0.883, 0.883)

In [167]:
## Policy iteration

In [175]:

def compute_value_function(env, policy, gamma = 1.0):
    iters = 10000
    v = np.zeros(env.nS)
    eps = 1e-15
    for i in range(0, iters):
        prev_v = np.copy(v)
        for s in range(0,env.nS):
            policy_action = policy[s]
            v[s] = sum([p*(r+gamma*prev_v[s_]) for p,s_,r,_ in env.P[s][policy_action]])
        if np.sum(np.fabs(prev_v - v)) <=eps:
            break
    return v

def extract_policy(env, v, gamma = 1.0):
    policy = np.zeros(env.nS)
    for s in range(0,env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(0,env.nA):
            q_sa[a] = sum([p*(r+gamma*v[s_]) for p,s_,r,_ in env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def policy_iteration(env, gamma = 1.0):
    policy = np.random.choice(env.nA, size = env.nS)
    iters = 10000
    for i in range(0,iters):
        old_policy_v = compute_value_function(env, policy, gamma)
        new_policy = extract_policy(env, old_policy_v, gamma)
        if np.all(new_policy == policy):
            print('Policy iteration converged after ', i)
            break;
        policy = new_policy
    return policy


opt_policy = policy_iteration(env.env)


In [173]:
opt_policy

array([3., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 2., 0.,
       0., 0., 0., 2., 3., 3., 2., 0., 0., 0., 1., 0., 0., 2., 2., 0., 3.,
       0., 0., 2., 1., 3., 2., 0., 0., 0., 1., 3., 0., 0., 2., 0., 0., 1.,
       0., 0., 0., 0., 2., 0., 1., 0., 0., 1., 2., 1., 0.])

In [174]:
evaluate_policy(env,opt_policy,n=1000, gamma = 1.0)


(0.88, 0.88)